# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from matplotlib import pyplot as plt

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)
#print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
# File to Load:
file_path ='../WeatherPy/output_data/cities.csv' # csv file created in part I of homework 

# Read csv cities file and store into data frame:
city_csv = pd.read_csv(file_path)
city_csv.head(10)



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Jalu,29.03,21.55,83.30,31,0,10.33,LY,1600221349
1,1,Santa Cruz de La Palma,28.68,-17.76,75.00,88,100,13.87,ES,1600221350
2,2,Roblin,51.23,-101.36,51.08,72,100,11.50,CA,1600221350
3,3,Khatanga,71.97,102.50,51.69,76,100,6.76,RU,1600221350
4,4,Saskylakh,71.92,114.08,51.10,67,100,5.41,RU,1600221350
5,5,City of San Pedro,14.35,121.02,84.20,79,75,5.82,PH,1600221350
6,6,Mahébourg,-20.41,57.70,68.00,64,75,9.17,MU,1600221350
7,7,Taoudenni,22.68,-3.98,98.69,14,0,16.71,ML,1600221350
8,8,Albany,42.60,-73.97,51.01,68,11,2.30,US,1600221261
9,9,Avarua,-21.21,-159.78,77.00,69,53,16.11,CK,1600221351


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Determan maximum humidity level in the city data frame:
maximum_humidity = city_csv['Humidity'].max()
maximum_humidity

100

In [22]:
# Store 'Lat' and 'Lng' into  locations: 
city_csv = city_csv.dropna() # drop NaN values
locations = city_csv[['Lat', 'Lng']].astype(float)
#print(locations)

# Convert humidity level to float and store:
humidity_rate = city_csv['Humidity'].astype(float)
#humidity_rate

In [23]:
# Create a humidity heat map layer:
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights = humidity_rate, 
                                 dissipating = False, max_intensity = maximum_humidity,
                                 point_radius = 1)


# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius = 1

fig.add_layer(heat_layer)

plt.savefig('Images/humidity heat map layer.png')
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
# Create new data frame with requested weather conditions:
city_csv_narrow = city_csv[(city_csv['Max Temp'] < 96) &
                             (city_csv['Max Temp'] > 55) &
                             (city_csv['Wind Speed'] < 10) &
                             (city_csv['Cloudiness'] < 15) &
                             (city_csv['Humidity'] < 30)]
# drop any rows will null values
city_csv_narrow = city_csv_narrow.dropna()

#city_csv_narrow
city_csv_narrow.count()


Unnamed: 0    11
City          11
Lat           11
Lng           11
Max Temp      11
Humidity      11
Cloudiness    11
Wind Speed    11
Country       11
Date          11
dtype: int64

In [25]:
# Set up lists to hold response info:
name=[]         # list for hotel name
city=[]         # list for hotel city
country=[]      # list for hotel country
lat=[]          # list for hotel latitude
lng=[]          # list for hotel longitude

# Loop through the list of cities and perform a request for data on each and add to corresponding lists:

count = 1 

print('Search Begin')     
print('-----------------------------')
for i in range(len(city_csv_narrow)) : 
   
    lati=str(city_csv_narrow.iloc[i, 2])    
    lngi=str(city_csv_narrow.iloc[i, 3])
    
    target_coordinates=lati + ', ' + lngi  
    target_search='Hotel'
    target_radius=5000
    target_type='lodging'

    # Set up a parameters dictionary:
    params={
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type': target_type,
        'key': g_key
    }

    # Base url:
    base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # Run a request using the above params dictionary:
    response = requests.get(base_url, params = params)
    #print(response.url)
    
    # Convert response to json:
    places_data = response.json()
    #print(json.dumps(places_data, indent=4, sort_keys=True))
    
    # Print user feedback: 
    print('Looking for Hotels near: ' + str(city_csv_narrow.iloc[i, 0]) + ' | Record ' + str(count) + ' of Total ' + str(len(city_csv_narrow)))
     
    # Add requested data to  result lists:
    try:                                             # ignore errors in response
        name.append(places_data['results'][0]['name'])
        city.append(city_csv_narrow.iloc[i, 0])
        country.append(city_csv_narrow.iloc[i, 1])
        lat.append(places_data['results'][0]['geometry']['location']['lat'])
        lng.append(places_data['results'][0]['geometry']['location']['lng'])
    except IndexError:
        print('No Hotel Near Location... Skipping...') 
        
        # in case one of the responses comes back with error "IndexError"
        # which could be no hotel near location 
  
    count = count + 1
    
# Print Ending Log Statement:
print('Complete')


Search Begin
-----------------------------
Looking for Hotels near: 67 | Record 1 of Total 11
No Hotel Near Location... Skipping...
Looking for Hotels near: 109 | Record 2 of Total 11
No Hotel Near Location... Skipping...
Looking for Hotels near: 180 | Record 3 of Total 11
No Hotel Near Location... Skipping...
Looking for Hotels near: 203 | Record 4 of Total 11
No Hotel Near Location... Skipping...
Looking for Hotels near: 262 | Record 5 of Total 11
No Hotel Near Location... Skipping...
Looking for Hotels near: 353 | Record 6 of Total 11
No Hotel Near Location... Skipping...
Looking for Hotels near: 391 | Record 7 of Total 11
No Hotel Near Location... Skipping...
Looking for Hotels near: 397 | Record 8 of Total 11
No Hotel Near Location... Skipping...
Looking for Hotels near: 419 | Record 9 of Total 11
No Hotel Near Location... Skipping...
Looking for Hotels near: 421 | Record 10 of Total 11
No Hotel Near Location... Skipping...
Looking for Hotels near: 515 | Record 11 of Total 11
No H

In [26]:
# Create a data frame from api call list:
hotel_dict={
    'City': city,
    'Country': country,
    'Lat': lat,
    'Lng': lng,
    'Hotel Name': name
}
hotel_df = pd.DataFrame(hotel_dict)
hotel_df.count()

City          0
Country       0
Lat           0
Lng           0
Hotel Name    0
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map


# Display figure
# Customize the size of the figure:
figure_layout={
    'width': '955px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout = figure_layout)

# Assign the marker layer to a variable:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map:
fig.add_layer(markers)


fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [29]:
 #Create a combined map:
fig=gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))